# Week 4

This is the set of programming assignments for week 4.

The tasks cover the following topics:
- reversible computing
- implementing quantum oracles

We recommend to solve the following katas before doing these assignments:
- DeutschJozsaAlgorithm (part 1)
- GroversAlgorithm (part 1)
- SolveSATWithGrover (parts 1 and 2)
- GraphColoring (except task 2.3)  

from https://github.com/Microsoft/QuantumKatas

## Part I. Marking oracles for classical functions

In this section the oracles are defined by their effect on computational basis states, as described in the lecture.

>All oracles have to be adjointable; adjoint variant of the operation can be specified either automatically (using "is Adj" in the operation signature) or manually (using "body" and "adjoint self"); see https://docs.microsoft.com/en-us/quantum/user-guide/using-qsharp/operations-functions#explicit-specialization-declarations for details on how to specify variants explicitly.

### Task 1.1. $|111...000...\rangle$ oracle
**Inputs:**  
1. N qubits in an arbitrary state $|x\rangle$ (input/query register)
2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

You are guaranteed that N is an even number.

**Goal:** Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), where $f(x) = 1$ if the first half of bits of $x$ are 1 and the second half of the bits are 0, and 0 otherwise.

Leave the query register in the same state it started in.

> Example: For N = 4, the oracle should mark the state 1100.

In [1]:
%kata T11_Test

open Microsoft.Quantum.Arrays;

operation Task11 (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // Note that your implementation has to be adjointable.
    let num = Length(queryRegister)/2;
    let controlArray = ConstantArray(num, true) + ConstantArray(num, false);
    (ControlledOnBitString(controlArray, X))(queryRegister, target);
}

Success!

### Task 1.2. Prefix matching oracle
**Inputs:**
1. N qubits in an arbitrary state $|x\rangle$ (input/query register)
2. a qubit in an arbitrary state $|y\rangle$ (target qubit)
3. a bit pattern of length K represented as an `Bool[]` ($1 \le K \le N$).  

k-th element of the pattern encodes the allowed states of the qubit `x[k]`:
`false` and `true` values represent states $|0\rangle$ and $|1\rangle$, respectively. 
A prefix of length K of a state $|x\rangle = |x_1, ..., x_n\rangle$ is the state of its first k qubits $|x_1, ..., x_K\rangle$.

**Goal:** Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), where $f(x) = 1$ if the prefix of length `K` of the bit string `x` equals the given pattern, and 0 if it does not.
    
Leave the query register in the same state it started in.

> For example, for N = 3 pattern of length 2 `[false, true]` would match two states: $|010\rangle$ and $|011\rangle$.

In [2]:
%kata T12_Test
open Microsoft.Quantum.Arrays;

operation Task12 (queryRegister : Qubit[], target : Qubit, pattern : Bool[]) : Unit is Adj {
    // Note that your implementation has to be adjointable.
    let prefix_len = Length(pattern);
    let prefix = queryRegister[0..prefix_len-1];
    (ControlledOnBitString(pattern, X))(prefix, target);
}

Success!

### Task 1.3. Regexp matching oracle
**Inputs:**
1. N qubits in an arbitrary state $|x\rangle$ (input/query register)
2. a qubit in an arbitrary state $|y\rangle$ (target qubit)
3. a bit pattern of length `N` represented as an `Int[]`.

k-th element of the pattern encodes the allowed states of the qubit `x[k]`:
0 and 1 values represent states $|0\rangle$ and $|1\rangle$, respectively, value $-1$ represents any state.

**Goal:** Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), where $f(x) = 1$ if the bit string x matches the given pattern, and 0 if it does not.

Leave the query register in the same state it started in.

> For example, pattern `[0, -1, 1]` would match two states: $|001\rangle$ and $|011\rangle$; pattern `[1, -1, -1]` would match four states: $|100\rangle$, $|101\rangle$, $|110\rangle$, and $|111\rangle$.

In [3]:
%kata T13_Test
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Logical;

operation Task13 (queryRegister : Qubit[], target : Qubit, pattern : Int[]) : Unit is Adj {
    // Note that your implementation has to be adjointable.
    // ...
    let predicate1 = EqualI(_, -1);
    let predicate2 = EqualI(_, 1);
    let predicate3 = EqualI(_, 0);

    let num_Q = Length(queryRegister);
    use q = Qubit[num_Q];
    within{
        for i in Where(predicate1, pattern){
            X(q[i]);
        }
        for i in Where(predicate2, pattern){
            CNOT(queryRegister[i], q[i]);
        }
        for i in Where(predicate3, pattern){
            X(q[i]);
            CNOT(queryRegister[i], q[i]);
        }
    }
    apply {
        Controlled X (q, target);
    }
}

Success!

### Task 1.4. Substring searching oracle
**Inputs:**
1. N qubits in an arbitrary state $|x\rangle$ (input/query register)
2. a qubit in an arbitrary state $|y\rangle$ (target qubit)
3. a bit string of length K represented as `Bool[]` ($K \le N$)

k-th element of the pattern encodes the states of the qubit : `false` and `true` values represent states $|0\rangle$ and $|1\rangle$, respectively. 
 
**Goal:** Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), where $f(x) = 1$ if the bit string $x$ contains the given bit string as a continuous substring, and 0 if it does not.
 
Leave the query register in the same state it started in.

> For example, bit string `[false, true, true]` would be found in states $|0111\rangle$ and $|0011\rangle$, but not in state $|1001\rangle$ or $|0101\rangle$; bit string `[false]` would be found in any state other than $|1...1\rangle$ (the string is allowed to occur in multiple places).

> Hint: use the logic of task 1.2 as a building block for this task.

In [4]:
%kata T14_Test

operation Task14 (queryRegister : Qubit[], target : Qubit, substring : Bool[]) : Unit is Adj {
    // Note that your implementation has to be adjointable.
    let substring_len = Length(substring);
    let query_len = Length(queryRegister);
    let check_len = query_len - substring_len + 1;
    use q = Qubit[check_len];
    within{
        for i in 0..check_len-1 {
            Task12 (queryRegister[i..i+substring_len-1], q[i], substring);
            X(q[i]);
        }
    }
    apply{
        Controlled X (q, target);
        X(target);
    }
}

Success!

### Task 1.5. Majority function on 5 qubits
**Inputs:**
1. 5 qubits in an arbitrary state $|x\rangle$ (input register)
2. a qubit in an arbitrary state $|y\rangle$ (output qubit)

**Goal:** transform state $|x\rangle|y\rangle$ into state $|x\rangle|y \oplus MAJ(x)\rangle$ ($\oplus$ is addition modulo 2), where MAJ is majority function on 5-bit vectors, defined as follows: $MAJ(x) = 1$ if 3 or more bits of $x$ are 1, and 0 otherwise.

In [5]:
%kata T15_Test

operation Task15 (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // Note that your implementation has to be adjointable.
    // This is a poor solution :/
    for i in 0..4{
        for j in i+1..4{
            for k in j+1..4{
                Controlled X([queryRegister[i], queryRegister[j], queryRegister[k]], target);
                for l in k+1..4{
                    Controlled X([queryRegister[i], queryRegister[j], queryRegister[k], queryRegister[l]], target);
                }                  
            }
        }     
    }
}       

Success!

## Part II. Oracle transformations and phase-flipping oracles

### Task 2.1. Arbitrary bit pattern phase-flipping oracle
**Inputs:**
1. N qubits in an arbitrary state $|x\rangle$ (input/query register)
2. a bit pattern of length `N` represented as `Bool[]`

**Goal:** Flip the phase of the register if it is in the state described by the given bit pattern (`false` and `true` values represent states $|0\rangle$ and $|1\rangle$, respectively). In this task you are not allowed to allocate extra qubits.

In [6]:
%kata T21_Test
open Microsoft.Quantum.Arrays;

operation Task21 (x : Qubit[], pattern : Bool[]) : Unit is Adj {
    // Note that your implementation has to be adjointable.
    // ...
    if(Head(pattern)){
        ControlledOnBitString(Rest(pattern), Z)(Rest(x), Head(x));
    }
    else{
        X(Head(x));
        ControlledOnBitString(Rest(pattern), Z)(Rest(x), Head(x));
        X(Head(x));
    }
}

Success!

>Tasks 2.2-2.4 require you to return an operation (oracle) constructed using another operation.
>See the reference solution for task 1.4 from the GroversAlgorithm kata for an example of defining an oracle based on another oracle and returning that oracle as an operation.
You will need to create a separate code cell to define the helper operation and use it in the cell with %kata magic.
Note that the operation you return has to be adjointable.

### Task 2.2. Conversion between different phase-flipping oracles
**Input:** A phase-flipping oracle of the form $|x\rangle → (-1)^{f(x)} |x\rangle$.  
**Output:** A phase-flipping oracle of the form $|x\rangle|b\rangle → (-1)^{b · f(x)} |x\rangle|b\rangle$, where $b$ is a one-qubit register, and $·$ is dot product of scalar values (this oracle only flips the phase of the register $x$ if $b$ is in 1 state). The operation you return should take two parameters: an array of qubits $|x\rangle$ and a single qubit $|b\rangle$.

> Hint: Note that the operation you're given (phaseOracle) has both adjoint and controlled variants.

In [7]:
operation PhaseFlippingOracle (phaseOracle : (Qubit[] => Unit is Adj+Ctl), register : Qubit[], b : Qubit) : Unit is Adj {
    Controlled phaseOracle([b], register);
}

PhaseFlippingOracle

In [8]:
%kata T22_Test

function Task22 (phaseOracle : (Qubit[] => Unit is Adj+Ctl)) : ((Qubit[], Qubit) => Unit is Adj) {
    // ...
        
    // Currently this function returns an existing oracle for the sake of being able to compile the code.
    // You will need to return your own oracle instead of Task11.
    return PhaseFlippingOracle(phaseOracle,_,_);
}

Success!

### Task 2.3. Conversion from phase-flipping oracle to marking oracle
**Input:** A phase-flipping oracle of the form $|x\rangle|b\rangle → (-1)^{b · f(x)} |x\rangle|b\rangle$.  
**Output:** A marking oracle of the form $|x\rangle|b\rangle → |x\rangle|b \oplus f(x)\rangle$.

The operation you return should take two parameters: an array of qubits $|x\rangle$ and a single qubit $|b\rangle$.

In [9]:
operation PhaseToMarkingOracle (phaseOracle : (Qubit[], Qubit) => Unit is Adj, register : Qubit[], b : Qubit) : Unit is Adj{
    H(b);
    phaseOracle(register, b);
    H(b);
}

PhaseToMarkingOracle

In [10]:
%kata T23_Test

function Task23 (phaseOracle : ((Qubit[], Qubit) => Unit is Adj)) : ((Qubit[], Qubit) => Unit is Adj) {
    // ...

    // Currently this function returns an existing oracle for the sake of being able to compile the code.
    // You will need to return your own oracle instead of Task11.
    return PhaseToMarkingOracle(phaseOracle,_,_);
}

Success!

### Task 2.4. Oracle with one extra state marked/unmarked
**Inputs:**
1. a marking oracle of the form $|x\rangle|b\rangle → |x\rangle|b \oplus f(x)\rangle$ ($x$ is an N-bit input)
2. a bit pattern of length N represented as `Bool[]` (`false` and `true` values represent states $|0\rangle$ and $|1$, respectively). 

**Output:** A marking oracle of the same form, which works as follows:
- for all $x \neq pattern$, $|x\rangle|b\rangle → |x\rangle|b \oplus f(x)\rangle$
- for $x = pattern$, $|x\rangle|b\rangle → |x\rangle|b \oplus f(x) \oplus 1\rangle$  

(i.e., if the input oracle marked the pattern as the solution, the output oracle will not mark it, and vice versa)

In [11]:
operation MarkingStateOracle (markingOracle : (Qubit[], Qubit) => Unit is Adj, pattern : Bool[], register : Qubit[], b : Qubit) : Unit is Adj {
    markingOracle(register, b);
    (ControlledOnBitString(pattern, X))(register, b);
}

MarkingStateOracle

In [12]:
%kata T24_Test

function Task24 (markingOracle : ((Qubit[], Qubit) => Unit is Adj), pattern : Bool[]) : ((Qubit[], Qubit) => Unit is Adj) {
    // ...
        
    // Currently this function returns an existing oracle for the sake of being able to compile the code.
    // You will need to return your own oracle instead of Task11.
    return MarkingStateOracle(markingOracle,pattern,_,_);
}

Success!